In [36]:
import torch
import math
import pandas as pd
from datasets import Dataset
from transformers import GPTNeoForCausalLM, GPT2Tokenizer, Trainer, TrainingArguments, AutoTokenizer

In [37]:
from huggingface_hub import notebook_login

notebook_login()

In [38]:
torch.cuda.empty_cache()
torch.manual_seed(42)
path = "D:/_Coding/Python/AI/Text Generators/AI Text and Code Generation with GPT Neo and Python/Transformers/gpt neo 125M"
tokenizer = AutoTokenizer.from_pretrained(path)
model = GPTNeoForCausalLM.from_pretrained(path).cuda()

loading configuration file D:/_Coding/Python/AI/Text Generators/AI Text and Code Generation with GPT Neo and Python/Transformers/gpt neo 125M\config.json
Model config GPTNeoConfig {
  "_name_or_path": "D:/_Coding/Python/AI/Text Generators/AI Text and Code Generation with GPT Neo and Python/Transformers/gpt neo 125M",
  "activation_function": "gelu_new",
  "architectures": [
    "GPTNeoForCausalLM"
  ],
  "attention_dropout": 0,
  "attention_layers": [
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local"
  ],
  "attention_types": [
    [
      [
        "global",
        "local"
      ],
      6
    ]
  ],
  "bos_token_id": 50256,
  "embed_dropout": 0,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": null,
  "layer_norm_epsilon": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "gpt_neo",

In [6]:
descriptions = pd.read_csv("descriptions_2.csv")
datasets = Dataset.from_pandas(descriptions)

In [7]:
print(datasets)

Dataset({
    features: ['Unnamed: 0', 'Description'],
    num_rows: 187938
})


In [8]:
print(len(datasets["Description"]))

187938


In [25]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'

In [26]:
train_encodings = tokenizer(datasets["Description"], truncation=True, padding=True)

In [2]:
# print(train_encodings[1])
decoded =tokenizer.decode(train_encodings["input_ids"][5])

NameError: name 'tokenizer' is not defined

In [1]:
type(decoded)
print(decoded)

NameError: name 'decoded' is not defined

In [ ]:
# block_size = tokenizer.model_max_length
block_size = 128

In [ ]:
import pickle

In [31]:
model_name = "gpt_neo_125M"
training_args = TrainingArguments(
    f"{model_name}-finetuned-hacks",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

In [34]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encodings,
    # eval_dataset=lm_datasets["validation"],
)

d:\_Coding\Python\AI\Text Generators\AI Text and Code Generation with GPT Neo and Python\venv\lib\site-packages\huggingface_hub\repository.py:705: FutureWarning: Creating a repository through 'clone_from' is deprecated and will be removed in v0.11.
  warnings.warn(
Cloning https://huggingface.co/ChronicTronic/gpt_neo_125M-finetuned-hacks into local empty directory.


In [35]:
trainer.train()

d:\_Coding\Python\AI\Text Generators\AI Text and Code Generation with GPT Neo and Python\venv\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3
  Number of trainable parameters = 125198592


  0%|          | 0/3 [00:00<?, ?it/s]

KeyError: 'Indexing with integers (to access backend Encoding for a given batch index) is not available when using Python based tokenizers'

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
trainer.push_to_hub()